In [ ]:
import os
import geopandas as gpd
import pandas as pd
import gc
import warnings

In [ ]:
# suppress nanmean of empty slice warning
warnings.filterwarnings("ignore", message="Mean of empty slice")

In [ ]:
%load_ext autoreload
import icebath as icebath
from icebath.core import bathy_workflow
%autoreload 2

In [ ]:
import dask
from dask.distributed import Client, LocalCluster
client = Client(processes=True, n_workers=1, threads_per_worker=1, memory_limit='14GB', dashboard_address=':8787')
client

In [ ]:
# set up variables to run
# inpardir = '/Users/jessica/projects/bathymetry_from_bergs/DEMs/2m/'
# fjord = 'JI'
# metastr='_mdf'

inpardir = '/Volumes/Keeper/bathy_from_bergs_Kane_imagery_2021/KaneWV02-not-run/'
# inpardir = '/Users/jessica/projects/bathymetry_from_bergs/DEMs/KaneWV03/'
fjord="KB"
metastr="_meta"

outdir = "/Users/jessica/projects/bathymetry_from_bergs/results/"

In [ ]:
dirlist = [d for d in os.listdir(inpardir) if not d.startswith('.')]

In [ ]:
print(dirlist)

In [ ]:
def recursive_dir_size(path):
    size = 0

    for x in os.listdir(path):
        if not os.path.isdir(os.path.join(path,x)):
            size += os.stat(os.path.join(path,x)).st_size
        else:
            size += recursive_dir_size(os.path.join(path,x))

    # convert to GB
    size = size / (1024.0**3)
    
    return size

In [ ]:
# run the bathymetry-from-iceberg-freeboard workflow on each dem (skipping larger dirs for now)
for indir in dirlist[:]:
#     print(indir)
    try:
#         print(recursive_dir_size(inpardir+indir))
        if recursive_dir_size(inpardir+indir) > 0.0 and recursive_dir_size(inpardir+indir) < 4.0:
            bathy_workflow.run_workflow(inpardir+indir+'/', fjord, outdir, indir[0:14] + "icebergs.gpkg", metastr=metastr, bitmask=True)
        else:
            continue
        gc.collect()
    except NotADirectoryError:
#         if recursive_dir_size(inpardir+indir) < 0.75:
        bathy_workflow.run_workflow(inpardir+'/', fjord, outdir, "JIicebergs.gpkg", metastr=metastr)
        break
#         else:
#             continue        

In [ ]:
reslist = [f for f in os.listdir(outdir) if f.endswith('.gpkg')]

In [ ]:
print(reslist)

In [ ]:
# combine the geopackages into one geodataframe
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(outdir+res) for res in reslist],
                                 ignore_index=True), crs=gpd.read_file(outdir+reslist[0]).crs)

In [ ]:
# remove rows with negative draft values
gdf = gdf[gdf.filtered_draft_med > 0]

In [ ]:
len(gdf)

In [ ]:
gdf

In [ ]:
gdf.bmach_errbed.min()

In [ ]:
gdf.filtered_draft_mad.values

In [ ]:
outfn = fjord + "_combined_results.gpkg"
gdf.to_file(outdir+outfn, driver="GPKG")

In [ ]:
from icebath.utils import plot as ibplot

In [ ]:
ibplot.meas_vs_infer_fig(gdf, save=False)